# met_site_data_creation_impact_scaled.py

This script will create cBioPortal timeline files for the `Diagnosis` tracks (Primary, Lymph Node, and Metastasis)

The class `cBioPortalDiagnosisTimeline` will leverage the organ site mapping that is used in the MSK-MET paper (20 sites of disease)




## Load Libraries

In [1]:
import sys  
sys.path.insert(0, '../mappings')
sys.path.insert(0, '../analysis')
sys.path.insert(0, '../')
sys.path.insert(0, '/mind_data/fongc2/cdm-utilities/minio_api/')
sys.path.insert(0, '/mind_data/fongc2/cdm-utilities/')
sys.path.insert(0, '/mind_data/fongc2/diagnosis_event_abstraction_icd/cbioportal/')


import os
import pandas as pd
from minio_api import MinioAPI
import constants_o_sites as const
from organ_mapping_analysis import OrganMappingAnalysisRND
from organ_mapping_rdn_processing import MetastaticSpreadMappingRND
from dx_cbioportal_timeline import cBioPortalDiagnosisTimeline


from dotenv import load_dotenv, find_dotenv, dotenv_values
from utils import set_debug_console, print_df_without_index, drop_cols, mrn_zero_pad, convert_to_int, read_minio_api_config


In [2]:
set_debug_console()

## Test object
This section tests the class that performs all of the tasks at `Minio Setup` and below

In [11]:
# Filenames (Minio objects) required to create "Diagnosis" timeline files for cBioPortal
fname_minio_env = '/mind_data/fongc2/cdm-utilities/minio_env.txt'
fname_cbio = 'cbioportal/mskimpact_ids.tsv'
fname_dx = 'diagnosis/table_diagnosis_clean.csv'
fname_demo = 'demographics/ddp_demographics.tsv'
fname_impact_map = 'summary/IMPACT_Darwin_Sample_Summary.tsv'
fname_rad_pred = 'radiology/aacr_predictions.csv'
config_cols = None

TBD on config_cols... There are way too many hardcoded column names and values in this class that maybe should be in some global file.

In [4]:
obj_dx_timeline = cBioPortalDiagnosisTimeline(fname_minio_config=fname_minio_env, 
                                              config_cols=config_cols, 
                                              const_organ_map=const, 
                                              fname_impact_map=fname_impact_map, 
                                              fname_demo=fname_demo, 
                                              fname_dx=fname_dx, 
                                              fname_cbio=fname_cbio, 
                                              fname_rad_pred=fname_rad_pred)

Loading mapping tables


In [5]:
df_prim = obj_dx_timeline.primary_dx_clean_and_merge(fname_save='/mind_data/msk-mind-datahub/msk-mind/datahub/mskimpact_test_june/data_timeline_diagnosis1.txt')

In [23]:
df_prim.head(50);

In [7]:
df_mets = obj_dx_timeline.metastatic_clean_and_merge(fname_save='/mind_data/msk-mind-datahub/msk-mind/datahub/mskimpact_test_june/data_timeline_indication_of_mets1.txt')

In [24]:
df_mets.head(10);

In [8]:
df_ln = obj_dx_timeline.lymph_node_clean_and_merge(fname_save='/mind_data/msk-mind-datahub/msk-mind/datahub/mskimpact_test_june/data_timeline_ln1.txt')

In [25]:
df_ln.head(10);

## Minio Setup

In [3]:
# env = load_dotenv('/mind_data/fongc2/cdm-utilities/minio_env.txt')
config = read_minio_api_config('/mind_data/fongc2/cdm-utilities/minio_env.txt')
dict_config = dict(config)

In [4]:
dict_config['MINIO_ENV']

'/mind_data/fongc2/.env'

In [5]:
load_dotenv(find_dotenv())
 
ACCESS_KEY = os.getenv("ACCESS_KEY")
SECRET_KEY = os.getenv('SECRET_KEY')
BUCKET = 'cdm-data'

ca_certs = '/mind_data/fongc2/certificate.crt'
url_port = "tllihpcmind6:9000"

In [6]:
obj_minio = MinioAPI(ACCESS_KEY=ACCESS_KEY, SECRET_KEY=SECRET_KEY, ca_certs=ca_certs, url_port=url_port)

In [7]:
obj_minio.print_list_objects(bucket_name=BUCKET, recursive=True, prefix="radio")

radiology/.ipynb_checkpoints/rad_emb_feat_imp-checkpoint.csv
radiology/.ipynb_checkpoints/radiology_features_embedding_avg_findings_all-checkpoint.csv
radiology/.ipynb_checkpoints/radiology_features_embedding_avg_findings_parsed-checkpoint.csv
radiology/.ipynb_checkpoints/radiology_features_embedding_avg_impressions-checkpoint.csv
radiology/aacr_predictions.csv
radiology/ddp_radiology_reports.tsv
radiology/ddp_radiology_reports.tsv.bkup
radiology/ddp_radiology_reports_char_limit.tsv
radiology/ddp_radiology_reports_full_parsed.tsv
radiology/feature_matrices/.ipynb_checkpoints/
radiology/feature_matrices/genie_lung_crc_train/.ipynb_checkpoints/genie_crc_lung_training_0604-checkpoint.csv
radiology/feature_matrices/genie_lung_crc_train/genie_crc_lung_train_adrenal_feature_matrix_0614.csv
radiology/feature_matrices/genie_lung_crc_train/genie_crc_lung_train_bone_feature_matrix_0614.csv
radiology/feature_matrices/genie_lung_crc_train/genie_crc_lung_train_cnsbrain_feature_matrix_0614.csv
radio

In [ ]:
df_test = obj_minio.load_obj(bucket_name=BUCKET, 
                               path_object='sandbox/fongc2/dx_met_timeline.txt', 
                               sep='\t')
df_test.head()

## Outputs

This script will create a cBioPortal formatted timeline file for metastatic site events
Events will be derived from four sources:
- Billing codes
- IMPACT sample info
- ICD-O registry
- Radiology reports (NLP predictions)

cBioPortal timeline file columns required:
- PATIENT_ID
- START_DATE
- STOP_DATE (For this file, can be left blank)
- EVENT_TYPE (Always "Diagnosis")
- SUBTYPE (For metastatic events, always "Metastasis")
- SOURCE (ICD Billing, MSK-IMPACT, Radiology Report)
- SOURCE_SPECIFIC (Imaging modality: CT, PET, MRI for radiology reports. Pathology for MSK-IMPACT. ICD Billing for billing codes.)
- DX_DESCRIPTION (ICD billing description, "Radiology Report", Pathology report specimen description (?))
- ANATOMIC_LOCATION (MSK-MET derived info)


In [75]:
col_order = ['PATIENT_ID', 
              'START_DATE', 
              'STOP_DATE',
              'EVENT_TYPE',
              'SUBTYPE',
              'SOURCE',
              'SOURCE_SPECIFIC',
              'DX_DESCRIPTION',
              'ANATOMIC_LOCATION']

col_order_prim = ['PATIENT_ID', 
                  'START_DATE', 
                  'STOP_DATE',
                  'EVENT_TYPE',
                  'SUBTYPE',
                  'SOURCE',
                  'DX_DESCRIPTION',
                  'AJCC',
                  'CLINICAL_GROUP', 
                  'PATH_GROUP']




### Filename for output

In [20]:
path = '../demo_data'
fname_save_anno = 'impact2017_met_site_annotations_impact_scaled.csv'
pathfilename_out = os.path.join(path, fname_save_anno)

## Load Data
- Demographics (for gender)
- ID mapping between dmp-id and mrn
- IMPACT sample data
- IMPACT sample summary (for DOP)
- Dx Timeline from Darwin
- Radiology report predictions

### Load cbioportal clinical data file

In [8]:
df_samples1 = obj_minio.load_obj(bucket_name=BUCKET, 
                               path_object='cbioportal/mskimpact_ids.tsv', 
                               sep='\t')

# For genie, fix ids
col_id = 'SAMPLE_ID'
col_id2 = 'DMP_ID'
col_sex = 'GENDER'
col_sample_type = 'SAMPLE_TYPE'
col_prim_site = 'PRIMARY_SITE'
col_met_site = 'METASTATIC_SITE'
col_icd_billing = 'ICD-9/10 Dx Code'


col_rep = {'patientId': col_id2,
           'sampleId': col_id}

cols_keep = ['SAMPLE_ID',
             'DMP_ID',
             'CANCER_TYPE',
             'SAMPLE_TYPE',
             'PRIMARY_SITE',
             'METASTATIC_SITE'
            ]

df_samples = df_samples1.rename(columns=col_rep)
# df_samples = df_samples1[list(col_rep.values())]
logic1 = df_samples[col_sample_type] == 'Metastasis'
df_samples = df_samples.loc[logic1, cols_keep]


In [12]:
df_samples1.head(1);


NameError: name 'df_samples1' is not defined

### Load demographics

In [10]:
df_demo = obj_minio.load_obj(bucket_name=BUCKET, 
                               path_object='demographics/ddp_demographics.tsv', 
                               sep='\t')

df_demo = mrn_zero_pad(df=df_demo, col_mrn='MRN')
df_demo = df_demo[['MRN', 'PT_BIRTH_DTE', 'GENDER']]

df_demo['PT_BIRTH_DTE'] = pd.to_datetime(df_demo['PT_BIRTH_DTE'])



In [11]:
df_demo.head(1);


### Load sample summary data

In [12]:
df_sample_summary = obj_minio.load_obj(bucket_name=BUCKET, 
                               path_object='summary/IMPACT_Darwin_Sample_Summary.tsv', 
                               sep='\t')

df_sample_summary = convert_to_int(df=df_sample_summary, list_cols=['MRN'])
df_sample_summary = mrn_zero_pad(df=df_sample_summary, col_mrn='MRN')

cols_keep = ['MRN', 'SAMPLE_ID', 'DMP_ID', 'DATE_OF_PROCEDURE_SURGICAL_EST']
df_sample_summary = df_sample_summary[cols_keep]

df_sample_summary['DATE_OF_PROCEDURE_SURGICAL_EST'] = pd.to_datetime(df_sample_summary['DATE_OF_PROCEDURE_SURGICAL_EST'])

In [13]:
df_sample_summary;

### Load ID mapping

In [14]:
df_id_map = obj_minio.load_obj(bucket_name=BUCKET, 
                               path_object='id_mapping/ddp_id_mapping_pathology.tsv', 
                               sep='\t')

df_id_map = mrn_zero_pad(df=df_id_map, col_mrn='MRN')
df_id_map = df_id_map[['MRN', 'DMP_ID']].drop_duplicates()
df_id_map = df_id_map[df_id_map['DMP_ID'].notnull()]



In [15]:
df_id_map.head(1);


### Load Dx timeline data

In [112]:
df_dx_timeline = obj_minio.load_obj(bucket_name=BUCKET, 
                               path_object='diagnosis/table_diagnosis_clean.csv', 
                               sep=',')

df_dx_timeline = mrn_zero_pad(df=df_dx_timeline, col_mrn='MRN')

df_dx_timeline['Diagnosis Date'] = pd.to_datetime(df_dx_timeline['Diagnosis Date'])
df_dx_timeline = df_dx_timeline[df_dx_timeline['IS_NON_CANCER'] == False]


In [13]:
df_dx_timeline.head(1);

NameError: name 'df_dx_timeline' is not defined

#### Tumor Registry (Add this to Primary Dx timeline)

In [114]:
cols_dx_treg = ['MRN', 'Diagnosis Date', 
           'IS_STAGE_IV_ICDO_SOLID', 'ICDO_IS_DISTANT_SOLID']
df_dx_treg = df_dx_timeline.loc[df_dx_timeline['Source'] == 'Tumor Registry']

dx_desc = df_dx_treg['ICD-O Histology Desc'] + ' | ' + df_dx_treg['ICD-O Site Desc']
source = df_dx_treg['ICD-O Histology Code'] + ' | ' + df_dx_treg['ICD-O Site Code']
df_dx_treg = df_dx_treg.assign(DX_DESCRIPTION=dx_desc)
df_dx_treg = df_dx_treg.assign(SOURCE=source)

cols_dict = {'Clinical Group': 'CLINICAL_GROUP',
             'Path Group': 'PATH_GROUP'}

cols_keep = ['MRN', 'Diagnosis Date', 'DX_DESCRIPTION', 'SOURCE', 'CLINICAL_GROUP', 'PATH_GROUP', 'AJCC']

df_dx_treg = df_dx_treg.rename(columns=cols_dict)
df_dx_treg = df_dx_treg[cols_keep]

# df_dx_treg_mets = df_dx_treg_mets.drop(columns=['IS_STAGE_IV_ICDO_SOLID', 'ICDO_IS_DISTANT_SOLID'])

In [14]:
df_dx_treg.head(1);


NameError: name 'df_dx_treg' is not defined

#### Billing codes
##### Metastatic disease

In [116]:
cols_dx_billing = ['MRN', 'ICD-9/10 Dx Code', 'Diagnosis Description', 'Diagnosis Date']

logic1 = df_dx_timeline['Diagnosis Type'] == 'METS'
logic2 = df_dx_timeline['Source'] == 'Billing'
logics = logic1 & logic2
df_dx_billing = df_dx_timeline.loc[logics, cols_dx_billing]


In [117]:
df_dx_billing.head(1);


##### Primary dx

In [118]:
logic1 = df_dx_timeline['Diagnosis Type'] == 'CA'
logic2 = df_dx_timeline['Source'] == 'Billing'
cols_keeps = ['MRN', 'Diagnosis Date']

logics = logic1 & logic2
df_dx_timeline_prim = df_dx_timeline.loc[logics].copy()
dx_desc = df_dx_timeline_prim['Diagnosis'] 
source = df_dx_timeline_prim['ICD-9/10 Dx Code']
df_dx_timeline_prim = df_dx_timeline_prim[cols_keeps]

df_dx_timeline_prim = df_dx_timeline_prim.assign(DX_DESCRIPTION=dx_desc)
df_dx_timeline_prim = df_dx_timeline_prim.assign(SOURCE=source)





In [119]:
df_dx_timeline_prim.head(1)

,MRN,Diagnosis Date,DX_DESCRIPTION,SOURCE
0,00000094,2016-09-19,"MALIGNANT MELANOMA OF SKIN, UNSPECIFIED",C43.9


### Load Radiology report met predictions

In [23]:
df_rad_pred = obj_minio.load_obj(bucket_name=BUCKET, 
                               path_object=fname_rad_pred, 
                               sep=',')
df_rad_pred['RADIOLOGY_PERFORMED_DATE'] = pd.to_datetime(df_rad_pred['RADIOLOGY_PERFORMED_DATE'])
df_rad_pred = df_rad_pred.drop(columns=['ACCESSION_NUMBER'])


In [15]:
df_rad_pred.head();


NameError: name 'df_rad_pred' is not defined

## Merge and clean input data
- Add date of birth and sex/gender to dataframes
- Add impact IDs
- Select important columns

### Sample data

In [37]:
df_samples_f = df_sample_summary.merge(right=df_samples, how='right', on=['SAMPLE_ID', 'DMP_ID'])
df_samples_f = df_samples_f.merge(right=df_demo, how='left', on='MRN')

age_dop = (df_samples_f['DATE_OF_PROCEDURE_SURGICAL_EST'] - df_samples_f['PT_BIRTH_DTE']).dt.days
df_samples_f = df_samples_f.assign(AGE_DX=age_dop)

df_samples_f = df_samples_f[df_samples_f['AGE_DX'].notnull()]
df_samples_f['AGE_DX'] = df_samples_f['AGE_DX'].astype(int) 
df_samples_f = df_samples_f.drop(columns=['MRN', 'DATE_OF_PROCEDURE_SURGICAL_EST', 'PT_BIRTH_DTE'])


In [16]:
print(df_samples_f.shape)
df_samples_f.head(1);


NameError: name 'df_samples_f' is not defined

### Billing code data

#### Mets and LN

In [120]:
df_dx_billing_f = df_dx_billing.merge(right=df_demo, how='left', on='MRN')
df_id_map = df_sample_summary[['DMP_ID', 'MRN']].drop_duplicates()
df_dx_billing_f = df_id_map.merge(right=df_dx_billing_f, how='right', on='MRN')

age_dx = (df_dx_billing_f['Diagnosis Date'] - df_dx_billing_f['PT_BIRTH_DTE']).dt.days
df_dx_billing_f = df_dx_billing_f.assign(AGE_DX=age_dx)

df_dx_billing_f = df_dx_billing_f.drop(columns=['Diagnosis Date', 'PT_BIRTH_DTE', 'MRN'])
df_dx_billing_f = df_dx_billing_f[df_dx_billing_f['DMP_ID'].notnull()]

In [17]:
df_dx_billing_f.head(1);

NameError: name 'df_dx_billing_f' is not defined

#### Primary dx

In [122]:
df_dx_billing_prim_f = df_dx_timeline_prim.merge(right=df_demo, how='left', on='MRN')
df_id_map = df_sample_summary[['DMP_ID', 'MRN']].drop_duplicates()
df_dx_billing_prim_f = df_id_map.merge(right=df_dx_billing_prim_f, how='right', on='MRN')

age_dx = (df_dx_billing_prim_f['Diagnosis Date'] - df_dx_billing_prim_f['PT_BIRTH_DTE']).dt.days
df_dx_billing_prim_f = df_dx_billing_prim_f.assign(START_DATE=age_dx)

df_dx_billing_prim_f = df_dx_billing_prim_f.drop(columns=['Diagnosis Date', 'PT_BIRTH_DTE', 'MRN'])
df_dx_billing_prim_f = df_dx_billing_prim_f[df_dx_billing_prim_f['DMP_ID'].notnull()]

df_dx_billing_prim_f = df_dx_billing_prim_f.rename(columns={'DMP_ID': 'PATIENT_ID'})


df_dx_billing_prim_f = df_dx_billing_prim_f.assign(STOP_DATE='')
df_dx_billing_prim_f = df_dx_billing_prim_f.assign(EVENT_TYPE='Diagnosis')
df_dx_billing_prim_f = df_dx_billing_prim_f.assign(SUBTYPE='ICD Billing')
df_dx_billing_prim_f = df_dx_billing_prim_f.assign(AJCC='')
df_dx_billing_prim_f = df_dx_billing_prim_f.assign(CLINICAL_GROUP='')
df_dx_billing_prim_f = df_dx_billing_prim_f.assign(PATH_GROUP='')

df_dx_billing_prim_f = df_dx_billing_prim_f[col_order_prim]


In [18]:
df_dx_billing_prim_f.head(1);

NameError: name 'df_dx_billing_prim_f' is not defined

### Tumor registry data

In [124]:
df_dx_treg_f = df_dx_treg.merge(right=df_demo, how='left', on='MRN')
df_id_map = df_sample_summary[['DMP_ID', 'MRN']].drop_duplicates()
df_dx_treg_f = df_id_map.merge(right=df_dx_treg_f, how='right', on='MRN')

age_dx = (df_dx_treg_f['Diagnosis Date'] - df_dx_treg_f['PT_BIRTH_DTE']).dt.days
df_dx_treg_f = df_dx_treg_f.drop(columns=['MRN', 'GENDER', 'Diagnosis Date', 'PT_BIRTH_DTE'])
df_dx_treg_f = df_dx_treg_f[df_dx_treg_f['DMP_ID'].notnull()]

df_dx_treg_f = df_dx_treg_f.rename(columns={'DMP_ID': 'PATIENT_ID'})

df_dx_treg_f = df_dx_treg_f.assign(START_DATE=age_dx)
df_dx_treg_f = df_dx_treg_f.assign(STOP_DATE='')
df_dx_treg_f = df_dx_treg_f.assign(EVENT_TYPE='Diagnosis')
df_dx_treg_f = df_dx_treg_f.assign(SUBTYPE='Tumor Registry')
df_dx_treg_f = df_dx_treg_f.assign(SOURCE='Tumor Registry')

df_dx_treg_f = df_dx_treg_f[col_order_prim]


In [22]:
df_dx_treg_f.head(1);

NameError: name 'df_dx_treg_f' is not defined

In [21]:
pd.concat([df_dx_treg_f, df_dx_billing_prim_f], axis=0, sort=False).sort_values(by=['PATIENT_ID', 'START_DATE']);

NameError: name 'df_dx_treg_f' is not defined

### Radiology report predictions

In [77]:
dict_cols = {'Adrenal_Glands': 'ADRENAL_GLAND', 
             'Biliary_Tract': 'BILIARY_TRACT', 
             'Bone': 'BONE', 
             'Bowel': 'BOWEL', 
             'CNS_Brain': 'CNS_BRAIN', 
             'Liver': 'LIVER', 
             'Lung': 'LUNG', 
             'Lymph_Nodes': 'LYMPH', 
             'Mediastinum': 'MEDIASTINUM', 
             'Intra.Abdominal': 'INTRA_ABDOMINAL', 
             'Pleura': 'PLEURA'
            }
# missing: 'BLADDER_OR_URINARY_TRACT', 'BREAST', 'GENITAL_MALE', 'GENITAL_FEMALE', 'HEAD_AND_NECK', 'KIDNEY', 'OTHER', 'OVARY', 'PERIPHERAL_NERVOUS_SYSTEM', 'SKIN'
cols_organs = list(dict_cols.values()) + ['GENITAL_FEMALE', 'GENITAL_MALE']

df_id_map = df_sample_summary[['DMP_ID', 'MRN']].drop_duplicates()
df_rad_pred_f = df_id_map.merge(right=df_rad_pred, how='right', on='DMP_ID')
df_rad_pred_f = df_rad_pred_f.merge(right=df_demo, how='left', on='MRN')

age_dx = (df_rad_pred_f['RADIOLOGY_PERFORMED_DATE'] - df_rad_pred_f['PT_BIRTH_DTE']).dt.days
df_rad_pred_f = df_rad_pred_f.assign(AGE_DX=age_dx)

df_rad_pred_f = df_rad_pred_f.rename(columns=dict_cols)
GENITAL_MALE = (df_rad_pred_f['Reproductive_Organs'] == 1) & (df_rad_pred_f['GENDER'] == 'MALE')
GENITAL_FEMALE = (df_rad_pred_f['Reproductive_Organs'] == 1) & (df_rad_pred_f['GENDER'] == 'FEMALE')
df_rad_pred_f = df_rad_pred_f.assign(GENITAL_MALE=GENITAL_MALE)
df_rad_pred_f = df_rad_pred_f.assign(GENITAL_FEMALE=GENITAL_FEMALE)


cols_id = ['DMP_ID', 'AGE_DX', 'PROCEDURE_TYPE']



df_rad_pred_f_melt = pd.melt(frame=df_rad_pred_f, id_vars=cols_id, value_vars=cols_organs, var_name='ANATOMIC_LOCATION', value_name='value')
df_rad_pred_f_melt['value'] = df_rad_pred_f_melt['value'].astype(int)


df_rad_pred_f_melt = df_rad_pred_f_melt.rename(columns={'PROCEDURE_TYPE': 'SOURCE_SPECIFIC',
                                                       'AGE_DX': 'START_DATE',
                                                       'DMP_ID': 'PATIENT_ID',
                                                       })

df_rad_pred_f_melt = df_rad_pred_f_melt[df_rad_pred_f_melt['value'] == 1]
df_rad_pred_f_melt = df_rad_pred_f_melt.drop(columns=['value'])


df_rad_pred_f_melt = df_rad_pred_f_melt.assign(SOURCE='MSK-IMPACT')
df_rad_pred_f_melt = df_rad_pred_f_melt.assign(STOP_DATE='')
df_rad_pred_f_melt = df_rad_pred_f_melt.assign(EVENT_TYPE='Diagnosis')
df_rad_pred_f_melt = df_rad_pred_f_melt.assign(DX_DESCRIPTION='NLP Derived')
df_rad_pred_f_melt = df_rad_pred_f_melt.assign(SOURCE='Radiology Report')





#### Non-LN Organs

In [81]:
df_rad_pred_f_melt_vis = df_rad_pred_f_melt[~df_rad_pred_f_melt['ANATOMIC_LOCATION'].isin(['LYMPH'])]
df_rad_pred_f_melt_vis = df_rad_pred_f_melt_vis.assign(SUBTYPE='Metastasis')
df_rad_pred_f_melt_vis = df_rad_pred_f_melt_vis[col_order]

In [19]:
df_rad_pred_f_melt_vis.head();

NameError: name 'df_rad_pred_f_melt_vis' is not defined

#### Lymph node annotations

In [84]:
df_rad_pred_f_melt_ln = df_rad_pred_f_melt[df_rad_pred_f_melt['ANATOMIC_LOCATION'].isin(['LYMPH'])]
df_rad_pred_f_melt_ln = df_rad_pred_f_melt_ln.assign(SUBTYPE='Lymph Nodes')
df_rad_pred_f_melt_ln = df_rad_pred_f_melt_ln[col_order]

In [20]:
df_rad_pred_f_melt_ln.head();

NameError: name 'df_rad_pred_f_melt_ln' is not defined

## Load Mapping Objects

### Load mapping tables

In [12]:
# Load RDN mapping
obj_met_map_rdn = MetastaticSpreadMappingRND(path=const.pathname,
                                             fname_all_sites=const.fname_mapping_rdn_all_sites,
                                             fname_hematogenous=const.fname_mapping_rdn_hematogenous,
                                             fname_localext=const.fname_mapping_rdn_localext,
                                             fname_lymphatic=const.fname_mapping_rdn_lymphatic,
                                             fname_site_map=const.fname_mapping_rdn_site_map,
                                             fname_billing_map=const.fname_mapping_rdn_billing_map,
                                             fname_billing_code_dict=const.fname_mapping_rdn_to_billing_codes)


Loading mapping tables


In [13]:
const.pathname


'/mind_data/fongc2/organ-site-mapping/mappings'

### Load mapping object

In [14]:
# Load annoations object
obj_mapping = OrganMappingAnalysisRND(obj_met_map=obj_met_map_rdn)


In [15]:
obj_met_map_rdn.df_map_icd_mapping['tissue_icd_billing'].unique()


array(['ADRENAL_GLAND', 'BILIARY_TRACT', 'BLADDER_OR_URINARY_TRACT',
       'BONE', 'BOWEL', 'BREAST', 'CNS_BRAIN', 'GENITAL_MALE',
       'GENITAL_FEMALE', 'HEAD_AND_NECK', 'KIDNEY', 'LIVER', 'LUNG',
       'LYMPH', 'MEDIASTINUM', 'OTHER', 'OVARY',
       'PERIPHERAL_NERVOUS_SYSTEM', 'INTRA_ABDOMINAL', 'PLEURA', 'SKIN'],
      dtype=object)

## Apply mappings and format frames into cBioPortal timeline format
### Clinical Samples Table from cBioPortal

In [49]:
# Annotate IMPACT sample site data
df_met_sites_impact = obj_mapping.annotate_mapping_impact_met_samples(df_samples=df_samples_f, 
                                                                      col_primary_site=col_prim_site, 
                                                                      col_met_site=col_met_site, 
                                                                      label_dist_ln=True)

cols_keep = ['DMP_ID', 'AGE_DX', 'METASTATIC_SITE_BILLING_RDN', 'METASTATIC_SITE']
df_met_sites_impact = df_met_sites_impact[cols_keep]
df_met_sites_impact = df_met_sites_impact.assign(SOURCE='MSK-IMPACT')
df_met_sites_impact = df_met_sites_impact.assign(STOP_DATE='')
df_met_sites_impact = df_met_sites_impact.assign(EVENT_TYPE='Diagnosis')
df_met_sites_impact = df_met_sites_impact.assign(SOURCE='MSK-IMPACT')
df_met_sites_impact = df_met_sites_impact.assign(SOURCE_SPECIFIC='Pathology')

df_met_sites_impact = df_met_sites_impact.rename(columns={'DMP_ID': 'PATIENT_ID',
                                                          'METASTATIC_SITE_BILLING_RDN': 'ANATOMIC_LOCATION',
                                                          'AGE_DX': 'START_DATE',
                                                          'METASTATIC_SITE': 'DX_DESCRIPTION'
                                                         })


In [11]:
df_met_sites_impact.head()

NameError: name 'df_met_sites_impact' is not defined

#### Non-LN organs

In [51]:
df_met_sites_impact_vis = df_met_sites_impact[~df_met_sites_impact['ANATOMIC_LOCATION'].isin(['LYMPH'])]

df_met_sites_impact_vis = df_met_sites_impact_vis.assign(SUBTYPE='Metastasis')
df_met_sites_impact_vis = df_met_sites_impact_vis[col_order]

#### Lymph node annotations

In [53]:
df_met_sites_impact_ln = df_met_sites_impact[df_met_sites_impact['ANATOMIC_LOCATION'].isin(['LYMPH'])]
df_met_sites_impact_ln = df_met_sites_impact_ln.assign(SUBTYPE='Lymph Nodes')
df_met_sites_impact_ln = df_met_sites_impact_ln[col_order]

In [10]:
df_met_sites_impact_ln.head()

NameError: name 'df_met_sites_impact_ln' is not defined

### ICD Billing table 

In [ ]:
# Annotate diagnosis table of ICD billings with renzo's mapping of metastatic sites
df_met_sites_dx = obj_mapping.annotate_icd_billing_met_dx(df_dx_mets=df_dx_billing_f, 
                                                          col_icd_billing=col_icd_billing, 
                                                          col_sex=col_sex,
                                                          add_msk_met_anno=True)

df_met_sites_dx['Diagnosis Description'] = df_met_sites_dx['ICD-9/10 Dx Code'] + ' - ' + df_met_sites_dx['Diagnosis Description']
df_met_sites_dx = df_met_sites_dx.drop(columns=['ICD-9/10 Dx Code', 'GENDER', 'METASTATIC_SITE_RDN_MAP'])

df_met_sites_dx = df_met_sites_dx.assign(SOURCE='ICD Billing')
df_met_sites_dx = df_met_sites_dx.assign(STOP_DATE='')
df_met_sites_dx = df_met_sites_dx.assign(EVENT_TYPE='Diagnosis')
df_met_sites_dx = df_met_sites_dx.assign(SOURCE_SPECIFIC='ICD Billing')

df_met_sites_dx = df_met_sites_dx.rename(columns={'DMP_ID': 'PATIENT_ID',
                                                  'tissue_icd_billing': 'ANATOMIC_LOCATION',
                                                  'AGE_DX': 'START_DATE',
                                                  'Diagnosis Description': 'DX_DESCRIPTION'
                                                  }
                                        )
df_met_sites_dx

In [ ]:
print(df_dx_billing_f.shape)
print(df_met_sites_dx.shape)


#### Non-LN organs

In [ ]:
df_met_sites_dx_vis = df_met_sites_dx[~df_met_sites_dx['ANATOMIC_LOCATION'].isin(['LYMPH', 'OTHER'])]

df_met_sites_dx_vis = df_met_sites_dx_vis.assign(SUBTYPE='Metastasis')
df_met_sites_dx_vis = df_met_sites_dx_vis[col_order]

In [ ]:
df_met_sites_dx_vis.head()

#### Lymph node annotations

In [ ]:
df_met_sites_dx_ln = df_met_sites_dx[df_met_sites_dx['ANATOMIC_LOCATION'].isin(['LYMPH', 'OTHER'])]

df_met_sites_dx_ln = df_met_sites_dx_ln.assign(SUBTYPE='Lymph Nodes')
df_met_sites_dx_ln = df_met_sites_dx_ln[col_order]

In [ ]:
# df_met_rdn_anno.head(50)
# t = df_met_rdn_anno.groupby('DMP_ID')['CANCER_TYPE'].nunique()
# df_met_rdn_anno[df_met_rdn_anno['DMP_ID'].isin(t[t > 1].index)].head(50)

df_met_sites_dx_ln.head()

## Merge dataframes

In [ ]:
df_mets_f = pd.concat([df_met_sites_impact, df_met_sites_dx_vis], axis=0, sort=False)
df_mets_f = df_mets_f.sort_values(by=['PATIENT_ID', 'START_DATE', 'ANATOMIC_LOCATION']).reset_index(drop=True)

In [ ]:
df_mets_f.head(50)

## Save annotations

In [ ]:
# Save RDN annotations
# df_mets_f.to_csv(pathfilename_out, index=False)
obj_minio.save_obj(df=df_mets_f, bucket_name=BUCKET, path_object='sandbox/fongc2/dx_met_timeline.txt', sep='\t')
